# Visualize the probability vs crop size for the data from Ullman et al. (2016) in one plot

This notebook creates one single plot for the data from Ullman et al. (2016) and visualizes the MIRC search. In the main body of the manuscript, it is Figure 3A. On the y-axis, the decreasing probability is shown and on the x-axis, the decreasing crop size (in real pixel space) and the image crops are displayed. The figure is saved to the folder that is specific to the experiment (it was created during the MIRC-search).

The plot is specific to the experimental conditions of considering all correct classes as one class and to stride1.

The notebook does not work for data other than the one from Ullman et
al. (2016).

## Your TODO

Please specify the path to the most top directory of your recognition
gap experiments, i.e. the parent directory of the analysis folder.

In [ ]:
import utils.util as util
import plot_utils
import data_npz_utils
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
path_to_recognition_gap_folder = "/gpfs01/bethge/home/jborowski/CHAM_recognition_gap/JOV_publication_git_bethgelab/recognition_gap/"

Please specify the name of the experiment folder whose data you would
like to visualize, i.e. where all the csv and npz files were stored to.

In [ ]:
exp_name = "exp_792020Ullman_list_as_one_classTrue_startidx0_stopidx9_Ullman4_v0"

In [ ]:
if ("Ullman4" not in exp_name) or ("list_as_one_classTrue" not in exp_name):
    raise Exception(
        "You are not using the experimental conditions that were plotted in Figure 3A. You need the data from Ullman et al., list_as_one=True and Ullman4.")

## Load libraries

In [ ]:

# custom imports
# enable importing from parent directory
sys.path.insert(1, path_to_recognition_gap_folder)

## Data

In [ ]:
exp_dir = os.path.join("..", "figures_and_data_from_experiments", exp_name)

In [ ]:
# list of paths to those directories whose data should be plotted
data_point_to_plot_list = data_npz_utils.get_list_of_data_points_to_plot(
    exp_dir)

## Plotting business

### Parameter

In [ ]:
# get dictionary of dictionary with the following information:
# imagenetnumber, wordnetID, word
imagenetnumber_wordnetID_word_dict = plot_utils.get_dict_of_dict_with_imagenet_number_wordnetID_word()

In [ ]:
# these are the datapoints (i.e. images specific to certain true
# class(es)) that should be plotted in the below order:
datapoints_list = ["glasses_INclass836", "suit_INclass834", "plane_INclass404"]

### Helper function

In [ ]:
def customize_label(img_identifier, imagenetnumber_wordnetID_word_dict):
    """Determine the label for the probability line.

    Args:
        img_identifier:                     string describing the datapoint, e.g. 'plane_INclass404'
        imagenetnumber_wordnetID_word_dict: dictionary with imagenetnumber, wordnetID, word

    Returns:
        probability_label:                  label for probability line
    """

    neuronID = "".join(s for s in img_identifier if s.isdigit())
    probability_label = imagenetnumber_wordnetID_word_dict[int(
        neuronID)]["word"]

    return probability_label

## Plot it!

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9, 2))

for numero, img_identifier in enumerate(datapoints_list):

    # get data
    img_class_dict = data_npz_utils.get_img_class_dict_all_data(
        data_point_to_plot_list, exp_dir, img_identifier)
    list_of_keys_for_plotting = plot_utils.get_list_of_keys_for_plotting(
        img_class_dict)

    # plot the performance
    crop_probability = [
        img_class_dict[cur_key].probability for cur_key in list_of_keys_for_plotting]
    probability_label = customize_label(
        img_identifier, imagenetnumber_wordnetID_word_dict)
    plot_utils.plot_probabilities(
        ax, crop_probability, probability_label, numero)

    # plot the crops under x-label ticks
    y_offset = numero * (plot_utils.size + 0.05)
    plot_utils.plot_crops_below_xaxis(
        fig,
        ax,
        img_class_dict,
        y_offset=y_offset,
        space_between_ticks=0.078,
        color_counter=numero,
        list_of_keys_for_plotting=list_of_keys_for_plotting)

plot_utils.plot_recognition_criterion(ax, crop_probability)

# axes
size_in_real_pixels_list = util.get_size_in_original_px_space_list()
orig_px_size = size_in_real_pixels_list[: len(crop_probability)]
plot_utils.customize_axes(ax, crop_probability, 1.02, orig_px_size)
plot_utils.hide_right_and_top_spine(ax)
legend = ax.legend(loc="lower left")

fig.savefig(
    os.path.join(
        exp_dir,
        "JOV_performance_plot.svg"),
    bbox_inches="tight")
fig.show()